# 🎯 JetX Progressive Training - 3 Aşamalı Model Eğitimi

**Bu notebook Progressive Training stratejisi ile JetX modelini eğitir.**

## 📊 Sorun:
Mevcut model **1.5 altı değerleri tahmin edemiyor** - Hep 1.5 üstü tahmin ediyor!

## 🎯 Çözüm: 3 Aşamalı Progressive Training

### AŞAMA 1: Regression-Only (200 epoch) ⏱️ ~45 dakika
- Model önce **değer tahmin etmeyi** öğrenir
- Learning Rate: 0.0003 (yüksek - hızlı öğrenme)
- Batch Size: 16
- Loss Weight: Regression 100%

### AŞAMA 2: Threshold Fine-Tuning (150 epoch) ⏱️ ~40 dakika
- **1.5 altı/üstü ayrımını** keskinleştirir
- Learning Rate: 0.0001
- Batch Size: 8
- Class Weight: **25x** (1.5 altı için)
- Loss Weight: Regression 40%, Threshold 60%

### AŞAMA 3: Full Model Fine-Tuning (150 epoch) ⏱️ ~60 dakika
- **Tüm output'ları** birlikte optimize eder
- Learning Rate: 0.00005
- Batch Size: 4
- Class Weight: **30x** (1.5 altı için)
- Loss Weight: Regression 50%, Classification 10%, Threshold 40%

## 🎯 Hedefler:
- 🔴 **1.5 Altı Doğruluk**: %75-80%+
- 🟢 **1.5 Üstü Doğruluk**: %80-85%+
- 💰 **Para Kaybı Riski**: %20 altı
- 📊 **MAE**: < 2.0

---

## 📦 Adım 1: Hazırlık (2-3 dakika)

Aşağıdaki butona tıklayın. Kütüphaneler yüklenecek ve proje hazırlanacak.

In [ ]:
#@title ▶️ Hazırlık - Buraya tıklayın { display-mode: "form" }

print("📦 Kütüphaneler yükleniyor...")
!pip install -q tensorflow scikit-learn pandas numpy scipy joblib matplotlib seaborn tqdm

print("\n📥 Proje GitHub'dan indiriliyor...")
import os
if os.path.exists('jetxpredictor'):
    !rm -rf jetxpredictor
!git clone https://github.com/onndd/jetxpredictor.git
%cd jetxpredictor

print("\n✅ Hazırlık tamamlandı!")
print("\n➡️ Şimdi bir sonraki hücreye geçin (aşağıdaki ▶️ butonuna tıklayın)")

## 🔥 Adım 2: Progressive Training (2.5 saat)

Aşağıdaki butona tıklayın. 3 aşamalı model eğitimi başlayacak.

**⚠️ ÖNEMLİ:**
- Bu işlem 2.5 saat sürecek
- Bilgisayarınızı kapatmayın!
- Her 5 epoch'ta 1.5 altı/üstü doğruluk oranlarını göreceksiniz
- 3 aşama sırayla otomatik çalışacak

In [ ]:
#@title ▶️ Progressive Training Başlat - Buraya tıklayın { display-mode: "form" }

print("🎯 PROGRESSIVE TRAINING Başlatılıyor...")
print("⏱️ Tahmini süre: 2.5 saat (GPU ile)")
print("📊 İlerlemeyi aşağıda görebilirsiniz\n")
print("="*70)

!python notebooks/jetx_PROGRESSIVE_TRAINING.py

print("\n" + "="*70)
print("✅ Progressive Training tamamlandı!")
print("\n➡️ Şimdi bir sonraki hücreye geçin (aşağıdaki ▶️ butonuna tıklayın)")

## 💾 Adım 3: Modeli İndir

Aşağıdaki butona tıklayın. Eğitilmiş model dosyaları indirilecek.

In [ ]:
#@title ▶️ Dosyaları İndir - Buraya tıklayın { display-mode: "form" }

from google.colab import files
import os

print("💾 Dosyalar indiriliyor...\n")

model_files = [
    'jetx_progressive_final.h5',
    'scaler_progressive.pkl',
    'progressive_model_info.json'
]

for file in model_files:
    if os.path.exists(file):
        print(f"📥 {file} indiriliyor...")
        files.download(file)
    else:
        print(f"⚠️ {file} bulunamadı!")

print("\n✅ Tüm dosyalar bilgisayarınıza indirildi!")
print("\n📁 İndirilen dosyalar:")
print("   • jetx_progressive_final.h5 (Ana model)")
print("   • scaler_progressive.pkl (Normalizer)")
print("   • progressive_model_info.json (Model bilgileri)")
print("\n➡️ Bu dosyaları lokal projenizin 'models/' klasörüne kopyalayın")
print("\n🎉 Tamamlandı! Artık yeni modeli kullanabilirsiniz.")

---

## 📊 Progressive Training Detayları

### 🎯 Neden Progressive Training?

**Sorun:** Mevcut model 1.5 altı değerleri tahmin edemiyor!
- Eğitim sırasında hiç 1.5 altı tahmin yok
- Model "kolaya kaçıyor" (lazy learning)
- Learning rate çok düşük (0.00005)
- Class weight yetersiz (7x)

**Çözüm:** 3 Aşamalı Progressive Training

### 📋 Aşama Detayları:

#### AŞAMA 1: Regression-Only (200 epoch)
```
✅ Learning Rate: 0.0003 (6x artış!)
✅ Batch Size: 16 (daha hızlı)
✅ Model: 8-10M parametre (basitleştirilmiş)
✅ Loss: Sadece regression (değer öğrenme)
```
**Amaç:** Model önce sayısal değer tahmin etmeyi öğrenir

#### AŞAMA 2: Threshold Fine-Tuning (150 epoch)
```
✅ Learning Rate: 0.0001
✅ Batch Size: 8
✅ Class Weight: 25x (1.5 altı için)
✅ Loss Weight: Regression 40%, Threshold 60%
✅ Ultra Focal Loss (gamma=4.0)
```
**Amaç:** 1.5 altı/üstü ayrımını keskinleştir

#### AŞAMA 3: Full Model Fine-Tuning (150 epoch)
```
✅ Learning Rate: 0.00005 (çok hassas)
✅ Batch Size: 4
✅ Class Weight: 30x (1.5 altı için)
✅ Loss Weight: Reg 50%, Cls 10%, Thr 40%
```
**Amaç:** Tüm output'ları birlikte optimize et

### 🎯 Beklenen Sonuçlar:

| Metrik | Hedef | Mevcut Model |
|--------|-------|-------------|
| 1.5 Altı Doğruluk | %75-80%+ | %0-3% ❌ |
| 1.5 Üstü Doğruluk | %80-85%+ | %95%+ ✅ |
| Para Kaybı Riski | < %20 | %96-100% ❌ |
| MAE | < 2.0 | - |

### 📁 Kullanım:

1. İndirilen dosyaları lokal projenize kopyalayın:
   ```bash
   # macOS/Linux
   cp jetx_progressive_final.h5 ~/Desktop/jetxpredictor/models/jetx_model.h5
   cp scaler_progressive.pkl ~/Desktop/jetxpredictor/models/scaler.pkl
   ```

2. Streamlit uygulamasını yeniden başlatın:
   ```bash
   streamlit run app.py
   ```

3. Artık model 1.5 altı değerleri tahmin edebilecek!

---

## ⚠️ Önemli Notlar:

- 🚨 Bu model %100 doğru değildir
- 💰 Yine de para kaybedebilirsiniz
- 🎯 1.5x kritik eşiktir
- 🛡️ Rolling mod en güvenlidir (%80+ güven)
- ⚡ Aggressive mod çok risklidir

**Sorumlu oynayın!** 🎲